#  PROPHET

In [2]:
%%time
import pandas as pd
from prophet import Prophet

# Загрузка исторических данных о продажах из Excel/CSV-файла
data = pd.read_csv('C:/Users/bondarenKovv/Desktop/Python/Sales.csv',sep=';',low_memory=False)

# Преобразование формата даты
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)
data = data[(data['Дата'] >= '2021-01-01')]

# Подготовка данных для применения модели Prophet
data = data.rename(columns={'Дата': 'ds', 'Номер Магазина': 'store', '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'})
data['y'] = data['y'].str.replace(',' , '.').astype(float)

# Создание пустого DataFrame для хранения прогнозов
all_forecasts = pd.DataFrame()

# Словарь с параметрами модели для каждого магазина, включая значение по умолчанию
model_params = {'default': {'changepoint_prior_scale': 0.2, 'seasonality_prior_scale': 30},
                141: {'seasonality_prior_scale': 100},
                142: {'seasonality_prior_scale': 100},
                121: {'seasonality_prior_scale': 50}}  # Указываем параметр seasonality_prior_scale для магазина 141,121,142

# Группировка данных по магазинам и применение модели Prophet для каждого магазина
for store in data['store'].unique():
    store_data = data[data['store'] == store]
    
    if store in model_params:
        params = model_params[store]
    else:
        params = model_params['default']

    # Создание и обучение модели Prophet для каждого магазина с уточненными параметрами
    model = Prophet(**params)  # Используем параметры из словаря
    model.add_country_holidays(country_name='RU')  
    model.fit(store_data[['ds','y'] + ['store']])

    # Создание DataFrame для прогноза на N дней вперед
    future = model.make_future_dataframe(periods=40)
    forecast = model.predict(future)
    forecast['store'] = store  # Добавляем информацию о магазине
    
    # Добавление прогноза для текущего магазина к общему DataFrame
    all_forecasts = pd.concat([all_forecasts, forecast])

# Сохранение результатов в новом Excel-файле
all_forecasts.to_excel('forecast_next_month.xlsx', index=False)
'''1. changepoint_prior_scale: Этот параметр отвечает за гибкость модели в определении точек 
изменения тренда. Чем меньше значение, тем более гибкой будет модель, воспринимая меньшие изменения 
как точки изменения тренда. Установка этого параметра на низкий уровень позволяет модели более гибко 
реагировать на изменения в данных и следовать им более близко.
2. seasonality_prior_scale: Этот параметр контролирует насколько модель склонна к изучению и предсказыванию 
сезонных изменений. Если установить этот параметр на более высокое значение, то модель будет более склонна 
видеть и учитывать сезонные особенности в данных.'''

16:47:36 - cmdstanpy - INFO - Chain [1] start processing
16:47:37 - cmdstanpy - INFO - Chain [1] done processing
16:47:37 - cmdstanpy - INFO - Chain [1] start processing
16:47:38 - cmdstanpy - INFO - Chain [1] done processing
16:47:38 - cmdstanpy - INFO - Chain [1] start processing
16:47:38 - cmdstanpy - INFO - Chain [1] done processing
16:47:39 - cmdstanpy - INFO - Chain [1] start processing
16:47:39 - cmdstanpy - INFO - Chain [1] done processing
16:47:40 - cmdstanpy - INFO - Chain [1] start processing
16:47:40 - cmdstanpy - INFO - Chain [1] done processing
16:47:41 - cmdstanpy - INFO - Chain [1] start processing
16:47:41 - cmdstanpy - INFO - Chain [1] done processing
16:47:41 - cmdstanpy - INFO - Chain [1] start processing
16:47:42 - cmdstanpy - INFO - Chain [1] done processing
16:47:42 - cmdstanpy - INFO - Chain [1] start processing
16:47:42 - cmdstanpy - INFO - Chain [1] done processing
16:47:43 - cmdstanpy - INFO - Chain [1] start processing
16:47:43 - cmdstanpy - INFO - Chain [1]

16:48:39 - cmdstanpy - INFO - Chain [1] done processing
16:48:39 - cmdstanpy - INFO - Chain [1] start processing
16:48:40 - cmdstanpy - INFO - Chain [1] done processing
16:48:40 - cmdstanpy - INFO - Chain [1] start processing
16:48:41 - cmdstanpy - INFO - Chain [1] done processing
16:48:41 - cmdstanpy - INFO - Chain [1] start processing
16:48:41 - cmdstanpy - INFO - Chain [1] done processing
16:48:42 - cmdstanpy - INFO - Chain [1] start processing
16:48:42 - cmdstanpy - INFO - Chain [1] done processing
16:48:43 - cmdstanpy - INFO - Chain [1] start processing
16:48:43 - cmdstanpy - INFO - Chain [1] done processing
16:48:44 - cmdstanpy - INFO - Chain [1] start processing
16:48:44 - cmdstanpy - INFO - Chain [1] done processing
16:48:44 - cmdstanpy - INFO - Chain [1] start processing
16:48:45 - cmdstanpy - INFO - Chain [1] done processing
16:48:45 - cmdstanpy - INFO - Chain [1] start processing
16:48:45 - cmdstanpy - INFO - Chain [1] done processing
16:48:46 - cmdstanpy - INFO - Chain [1] 

16:49:27 - cmdstanpy - INFO - Chain [1] done processing
16:49:27 - cmdstanpy - INFO - Chain [1] start processing
16:49:27 - cmdstanpy - INFO - Chain [1] done processing
16:49:28 - cmdstanpy - INFO - Chain [1] start processing
16:49:28 - cmdstanpy - INFO - Chain [1] done processing
16:49:28 - cmdstanpy - INFO - Chain [1] start processing
16:49:28 - cmdstanpy - INFO - Chain [1] done processing
16:49:29 - cmdstanpy - INFO - Chain [1] start processing
16:49:29 - cmdstanpy - INFO - Chain [1] done processing
16:49:29 - cmdstanpy - INFO - Chain [1] start processing
16:49:29 - cmdstanpy - INFO - Chain [1] done processing
16:49:30 - cmdstanpy - INFO - Chain [1] start processing
16:49:30 - cmdstanpy - INFO - Chain [1] done processing
16:49:30 - cmdstanpy - INFO - Chain [1] start processing
16:49:30 - cmdstanpy - INFO - Chain [1] done processing
16:49:30 - cmdstanpy - INFO - Chain [1] start processing
16:49:31 - cmdstanpy - INFO - Chain [1] done processing
16:49:31 - cmdstanpy - INFO - Chain [1] 

CPU times: total: 3min 50s
Wall time: 4min 9s


'1. changepoint_prior_scale: Этот параметр отвечает за гибкость модели в определении точек \nизменения тренда. Чем меньше значение, тем более гибкой будет модель, воспринимая меньшие изменения \nкак точки изменения тренда. Установка этого параметра на низкий уровень позволяет модели более гибко \nреагировать на изменения в данных и следовать им более близко.\n2. seasonality_prior_scale: Этот параметр контролирует насколько модель склонна к изучению и предсказыванию \nсезонных изменений. Если установить этот параметр на более высокое значение, то модель будет более склонна \nвидеть и учитывать сезонные особенности в данных.'

In [31]:
import pandas as pd                      #Дообучение модели на примере 3х сезонных магазинов
from prophet import Prophet

# Загрузка исторических данных о продажах из Excel/CSV-файла
data = pd.read_csv('C:/Users/bondarenKovv/Desktop/Python/Sales.csv', sep=';', low_memory=False)

# Преобразование формата даты
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)
data = data[(data['Дата'] >= '2021-01-01')]

# Подготовка данных для применения модели Prophet
data = data.rename(columns={'Дата': 'ds', 'Номер Магазина': 'store', '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'})
data['y'] = data['y'].str.replace(',' , '.').astype(float)

# Создание пустого DataFrame для хранения прогнозов
all_forecasts = pd.DataFrame()

# Список магазинов для проведения прогноза
selected_stores = [141,142,121]

# Словарь с параметрами модели для каждого магазина
model_params = {'default': {'changepoint_prior_scale': 0.2, 'seasonality_prior_scale': 30},
                141: {'seasonality_prior_scale': 100},
                142: {'seasonality_prior_scale': 100},
                121: {'seasonality_prior_scale': 50}}  # Указываем параметр seasonality_prior_scale для магазина 141, 121, 142

# Группировка данных по магазинам и применение модели Prophet только для выбранных магазинов
for store in selected_stores:
    store_data = data[data['store'] == store]

    if store in model_params:
        params = model_params[store]
    else:
        params = model_params['default']

    # Создание и обучение модели Prophet для каждого магазина с уточненными параметрами
    model = Prophet(**params)  # Используем параметры из словаря
    model.add_country_holidays(country_name='RU')
    model.fit(store_data[['ds', 'y']])

    # Создание DataFrame для прогноза на N дней вперед
    future = model.make_future_dataframe(periods=40)
    forecast = model.predict(future)
    forecast['store'] = store  # Добавляем информацию о магазине

    # Добавление прогноза для текущего магазина к общему DataFrame
    all_forecasts = pd.concat([all_forecasts, forecast])

# Сохранение результатов в новом Excel-файле
all_forecasts.to_excel('forecast_selected_stores.xlsx', index=False)

15:49:32 - cmdstanpy - INFO - Chain [1] start processing
15:49:32 - cmdstanpy - INFO - Chain [1] done processing
